In [1]:
!git clone https://github.com/folfcoder/recaptcha-dataset.git

Cloning into 'recaptcha-dataset'...
remote: Enumerating objects: 11754, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 11754 (delta 5), reused 1 (delta 0), pack-reused 11742
Receiving objects: 100% (11754/11754), 395.64 MiB | 30.52 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (11779/11779), done.


In [2]:
!unzip '/content/drive/Othercomputers/내 컴퓨터/Konkuk_ML/CV/실습1/submit_early/query.zip'

Archive:  /content/drive/Othercomputers/내 컴퓨터/Konkuk_ML/CV/실습1/submit_early/query.zip
   creating: query/
  inflating: query/query1.png        
  inflating: query/query10.png       
  inflating: query/query100.png      
  inflating: query/query11.png       
  inflating: query/query12.png       
  inflating: query/query13.png       
  inflating: query/query14.png       
  inflating: query/query15.png       
  inflating: query/query16.png       
  inflating: query/query17.png       
  inflating: query/query18.png       
  inflating: query/query19.png       
  inflating: query/query2.png        
  inflating: query/query20.png       
  inflating: query/query21.png       
  inflating: query/query22.png       
  inflating: query/query23.png       
  inflating: query/query24.png       
  inflating: query/query25.png       
  inflating: query/query26.png       
  inflating: query/query27.png       
  inflating: query/query28.png       
  inflating: query/query29.png       
  inflating

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import cv2
from glob import glob
import shutil
import os
import random
from sklearn.model_selection import train_test_split,StratifiedKFold
from scipy import signal as sg
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops
from skimage.feature import hog
from skimage import data, exposure
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import normalize,StandardScaler
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.mixture import GaussianMixture
import pickle
import joblib

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything()

In [4]:
shutil.rmtree('/content/recaptcha-dataset/Large/Mountain/')
shutil.rmtree('/content/recaptcha-dataset/Large/Other/')
os.remove('/content/recaptcha-dataset/Large/readme.txt')
total_img_path = sorted(glob('/content/recaptcha-dataset/Large/*'))

In [5]:
def make_label_dict(total_img_path):
    label_dict = dict()
    for idx,name in enumerate(total_img_path):
        tmp = name.split('/')[-1]
        label_dict[idx]  = tmp
    return label_dict
label_dict = make_label_dict(total_img_path)

In [6]:
def make_img_path(total_img_path,label_dict):
    df_list = list()
    for idx,path in enumerate(total_img_path):
        tmp_list = glob(path+'/*')
        df = pd.DataFrame(tmp_list,columns=['path'])
        df['label'] = idx
        df_list.append(df)
    result = pd.concat(df_list,axis=0,ignore_index=True)
    return result
total_df = make_img_path(total_img_path,label_dict)

In [7]:
class early_vision_classifier():
    def __init__(self):
        self.kmeans = None
        self.kmeans2 = None
        self.knn = None
        self.transformer = None
        self.transformer2 = None
        self.scaler = None
        self.scaler2 = None
        self.sift_scaler = None
        self.sift_scaler2 = None
        self.train = None

    ####################### PreProcessing ####################################
    def bgr_to_hsv(self,image):
        resized_image = cv2.resize(image, (120, 120))
        return cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)

    def bgr_to_gray(self,image):
        resized_image = cv2.resize(image, (120, 120))
        return cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    def noise_filter(self,image,filter_type='gaussian',kernel_size=(3,3),sigma=1):

        if filter_type == 'gaussian':
            return cv2.GaussianBlur(image, ksize=kernel_size, sigmaX=sigma)
        elif filter_type == 'median':
            return cv2.medianBlur(image, ksize=kernel_size[0])
        else:
            return  cv2.blur(image, ksize=kernel_size)

    def local_binary_pattern(self,gray):
        # LBP P-반지름 R-1이면 중심과 인접한 8개 pixel 고려하겠다 --> 2^8개의 feature 생김
        lbp = local_binary_pattern(gray, P=8, R=1,method='uniform')

        return lbp

    def SIFT(self,image):

        sift = cv2.xfeatures2d.SIFT_create()
        '''
        kp1, des1 = sift.detectAndCompute(image, None)    # des: (n, 128) kp1, dex1 = (keypoing,descriptor)
        '''
        step = 15
        keypoints = [cv2.KeyPoint(x, y, step)
                  for y in range(0, image.shape[0], step)
                  for x in range(0, image.shape[1], step)]

        kp1, des1 = sift.compute(image, keypoints)

        lbp_hist = []
        padded_image = cv2.copyMakeBorder(image, 7, 7, 7, 7, cv2.BORDER_REFLECT)
        lbp_featuremap = self.local_binary_pattern(padded_image)
        for kp in kp1:
            x = int(kp.pt[0])+7
            y = int(kp.pt[1])+7
            patch = lbp_featuremap[x-7:x+7,y-7:y+7]
            hist_lbp, bin_lbp = np.histogram(patch.ravel(), bins=10, range=(0, 9),density=True)
            lbp_hist.append(hist_lbp.ravel())
        np.vstack(lbp_hist)  

        return kp1,des1,lbp_hist

    def save_model(self,path):

        print('Save Start...')
        save_kmeans = joblib.dump(self.kmeans,path+'/kmeans.pkl')
        save_kmeans2 = joblib.dump(self.kmeans2,path+'/kmeans2.pkl')
        
        save_knn = joblib.dump(self.knn,path+'/knn.pkl')

        save_transformer = joblib.dump(self.transformer,path+'/transformer.pkl')
        save_transformer2 = joblib.dump(self.transformer2,path+'/transformer2.pkl')

        save_scaler = joblib.dump(self.scaler,path+'/scaler.pkl')
        save_scaler2 = joblib.dump(self.scaler2,path+'/scaler2.pkl')

        save_sift_scaler = joblib.dump(self.sift_scaler,path+'/sift_scaler.pkl')
        save_sift_scaler2 = joblib.dump(self.sift_scaler2,path+'/sift_scaler2.pkl')

        save_train = self.train.to_csv(path+'/train_feature.csv',encoding='utf-8')
        print('Save End...')

    def load_model(self,path):
        print('Load Start...')
        self.kmeans = joblib.load(path+'/kmeans.pkl')
        self.kmeans2 = joblib.load(path+'/kmeans2.pkl')
        
        self.knn = joblib.load(path+'/knn.pkl')

        self.transformer = joblib.load(path+'/transformer.pkl')
        self.transformer2 = joblib.load(path+'/transformer2.pkl')

        self.scaler = joblib.load(path+'/scaler.pkl')
        self.scaler2 = joblib.load(path+'/scaler2.pkl')

        self.sift_scaler = joblib.load(path+'/sift_scaler.pkl')
        self.sift_scaler2 = joblib.load(path+'/sift_scaler2.pkl')

        self.train = pd.read_csv(path+'/train_feature.csv')
        self.train = self.train.drop(columns='Unnamed: 0')
        print('Load End...')


    def fit(self,df,n_cluster,n_lbp,n_neighbors):
        df =df.reset_index()
        X = list()
        hist_X = list()
        y = list()

        tmp_X = list()
        tmp_lbp = list()

        for idx in range(len(df)):
            img = cv2.imread(df.loc[idx,'path'])
            img = self.bgr_to_hsv(img)
            img = img[:,:,2]
            label = df.loc[idx,'label']

            result = self.noise_filter(img)
            

            kp,des,lbp_hist = self.SIFT(result)
            y.append(label)
            tmp_X.append(des)
            tmp_lbp.append(lbp_hist)

        
        self.sift_scaler = StandardScaler()
        tmp_X_scale = self.sift_scaler.fit_transform(np.vstack(tmp_X))

        self.kmeans = KMeans(n_clusters=n_cluster,n_init='auto',random_state=42,algorithm='elkan')
        self.kmeans.fit(tmp_X_scale)
        
        tmp_X_scale = np.reshape(tmp_X_scale,np.array(tmp_X).shape)

        for feature in tmp_X_scale:
            pred = self.kmeans.predict(feature)
            bow_hist,_ = np.histogram(pred,bins=n_cluster,range=(0,n_cluster),density=True)
            X.append(bow_hist)

        
        
        bow = np.vstack(X)
        ####################################################
        # tfidf
        self.transformer = TfidfTransformer(smooth_idf=True)
        self.scaler = StandardScaler()
        t = self.transformer.fit_transform(bow).toarray()
        t = self.scaler.fit_transform(t)
        ####################################################

        result_df = pd.DataFrame(t)
        result_df.add_prefix('SIFT')
        
        self.sift_scaler2 = StandardScaler()
        tmp_lbp_scale = self.sift_scaler2.fit_transform(np.vstack(tmp_lbp))

        self.kmeans2 = KMeans(n_clusters=n_lbp,n_init='auto',random_state=42,algorithm='elkan')
        self.kmeans2.fit_predict(tmp_lbp_scale)

        tmp_lbp_scale = np.reshape(tmp_lbp_scale,np.array(tmp_lbp).shape)

        for feature in tmp_lbp_scale:
            pred = self.kmeans2.predict(feature)
            bow_hist,_ = np.histogram(pred,bins=n_lbp,range=(0,n_lbp),density=True)
            hist_X.append(bow_hist)

        bow = np.vstack(hist_X)
        ####################################################
        # tfidf
        self.transformer2 = TfidfTransformer(smooth_idf=True)
        self.scaler2 = StandardScaler()
        t = self.transformer2.fit_transform(bow).toarray()
        t = self.scaler2.fit_transform(t)
        ####################################################

        hist_result_df = pd.DataFrame(t)
        hist_result_df.add_prefix('LBP')

        result_df = pd.concat([result_df,hist_result_df],axis=1)
        result_df['label'] = y


        self.knn = KNeighborsClassifier(n_neighbors = n_neighbors,metric='cosine',n_jobs=-1)
        self.knn.fit(result_df.drop(columns='label'),result_df.loc[:,'label'])
        self.train = result_df

        return result_df

    def validation(self,df,n_cluster,n_lbp):
        df =df.reset_index()
        X = list()
        hist_X = list()
        y = list()

        tmp_X = list()
        tmp_lbp = list()

        for idx in range(len(df)):
            img = cv2.imread(df.loc[idx,'path'])
            img = self.bgr_to_gray(img)
            label = df.loc[idx,'label']

            result = self.noise_filter(img)
            
        
            kp,des,lbp_hist = self.SIFT(result)
            y.append(label)
            tmp_X.append(des)
            tmp_lbp.append(lbp_hist)

        tmp_X_scale = self.sift_scaler.transform(np.vstack(tmp_X))
        tmp_X_scale = np.reshape(tmp_X_scale,np.array(tmp_X).shape)


        for feature in tmp_X_scale:
            pred = self.kmeans.predict(feature)
            bow_hist,_ = np.histogram(pred,bins=n_cluster,range=(0,n_cluster),density=True)
            X.append(bow_hist)

        bow = np.vstack(X)
        ####################################################
        # tfidf
        t = self.transformer.transform(bow).toarray()
        t = self.scaler.transform(t)
        ####################################################
        
        result_df = pd.DataFrame(t)
        result_df.add_prefix('SIFT')
        
        tmp_lbp_scale = self.sift_scaler2.transform(np.vstack(tmp_lbp))
        tmp_lbp_scale = np.reshape(tmp_lbp_scale,np.array(tmp_lbp).shape)
        
        for feature in tmp_lbp_scale:
            pred = self.kmeans2.predict(feature)
            bow_hist,_ = np.histogram(pred,bins=n_lbp,range=(0,n_lbp),density=True)
            hist_X.append(bow_hist)

        bow = np.vstack(hist_X)
        ####################################################
        # tfidf
        t = self.transformer2.transform(bow).toarray()
        t = self.scaler2.transform(t)
        ####################################################

        hist_result_df = pd.DataFrame(t)
        hist_result_df.add_prefix('LBP')

        result_df = pd.concat([result_df,hist_result_df],axis=1)
        result_df['label'] = y

        classification = self.knn.predict(result_df.drop(columns=['label']))
        retrieval = self.knn.kneighbors(X=result_df.drop(columns=['label']), n_neighbors=10, return_distance=False)

        ret_label = list()
        for i in retrieval:
            ret_label.append(self.train.loc[i,'label'])
        
        classification_acc = accuracy_score(result_df['label'],classification)

        retrieval_acc = list()
        for ret,lab in zip(ret_label,result_df['label']):
            acc = (ret==lab).sum()
            retrieval_acc.append(acc/10)
        retrieval_acc = np.mean(retrieval_acc)

        return classification_acc,retrieval_acc

    def test(self,df,n_cluster,n_lbp):
        df =df.reset_index()
        X = list()
        hist_X = list()
        

        tmp_X = list()
        tmp_lbp = list()

        for idx in range(len(df)):
            img = cv2.imread(df.loc[idx,'path'])
            img = self.bgr_to_gray(img)           
        
            result = self.noise_filter(img)
                    
            kp,des,lbp_hist = self.SIFT(result)   
            tmp_X.append(des)
            tmp_lbp.append(lbp_hist)


        for feature in tmp_X:
            pred = self.kmeans.predict(feature)
            bow_hist,_ = np.histogram(pred,bins=n_cluster,range=(0,n_cluster),density=True)
            X.append(bow_hist)

        bow = np.vstack(X)
        ####################################################
        # tfidf
        t = self.transformer.transform(bow).toarray()
        t = self.scaler.transform(t)
        ####################################################
        
        result_df = pd.DataFrame(t)
        result_df.add_prefix('SIFT')
        
        
        for feature in tmp_lbp:
            pred = self.kmeans2.predict(feature)
            bow_hist,_ = np.histogram(pred,bins=n_lbp,range=(0,n_lbp),density=True)
            hist_X.append(bow_hist)

        bow = np.vstack(hist_X)
        ####################################################
        # tfidf
        t = self.transformer2.transform(bow).toarray()
        t = self.scaler2.transform(t)
        ####################################################

        hist_result_df = pd.DataFrame(t)
        hist_result_df.add_prefix('LBP')

        result_df = pd.concat([result_df,hist_result_df],axis=1)

        classification = self.knn.predict(result_df)

        retrieval = self.knn.kneighbors(X=result_df, n_neighbors=10, return_distance=False)
        ret_label = list()
        for i in retrieval:
            ret_label.append(self.train.loc[i,'label'].values)


        return classification,ret_label

In [8]:
total_test_img_path = sorted(glob('/content/query/*'))
total_test_df = pd.DataFrame(data=total_test_img_path,columns=['path'])
total_test_df

,path
0,/content/query/query1.png
1,/content/query/query10.png
2,/content/query/query100.png
3,/content/query/query11.png
4,/content/query/query12.png
...,...
95,/content/query/query95.png
96,/content/query/query96.png
97,/content/query/query97.png
98,/content/query/query98.png


In [9]:
def int2str(df,label_dict):
    tmp = df.copy()
    tmp.loc[:,'classification'] = tmp.loc[:,'classification'].apply(lambda x: label_dict[x])
    tmp.loc[:,'ret'] = df.loc[:,'ret'].apply(lambda x: [label_dict[label] for label in x])

    return tmp
#int2str(submit,label_dict)

In [10]:
classifier3 = early_vision_classifier()
classifier3.load_model('/content/drive/Othercomputers/내 컴퓨터/Konkuk_ML/CV/model/SIFT_LBP_COS_300')
n_cluster = 300
n_lbp = 100

Load Start...
Load End...


In [11]:
classification,ret_label = classifier3.test(total_test_df,n_cluster=n_cluster,n_lbp=n_lbp)
submit = total_test_df.copy() 
submit['classification'] = classification
submit['ret'] = ret_label
submit = int2str(submit,label_dict)
submit_classification = submit.drop(columns='ret')
submit_classification.loc[:,'path'] = submit_classification.loc[:,'path'].apply(lambda x: x.split('/')[-1])
submit_ret = submit.drop(columns='classification')
submit_ret[[0,1,2,3,4,5,6,7,8,9]] = submit_ret.loc[:,'ret'].to_list()
submit_ret = submit_ret.drop(columns='ret')
submit_ret.loc[:,'path'] = submit_ret.loc[:,'path'].apply(lambda x: x.split('/')[-1])

In [12]:
submit_classification

,path,classification
0,query1.png,Palm
1,query10.png,Palm
2,query100.png,Car
3,query11.png,Palm
4,query12.png,Palm
...,...,...
95,query95.png,Car
96,query96.png,Crosswalk
97,query97.png,Palm
98,query98.png,Hydrant


In [13]:
submit_ret

,path,0,1,2,3,4,5,6,7,8,9
0,query1.png,Palm,Palm,Palm,Palm,Palm,Traffic Light,Palm,Hydrant,Palm,Car
1,query10.png,Palm,Motorcycle,Palm,Palm,Traffic Light,Palm,Palm,Palm,Bus,Palm
2,query100.png,Car,Car,Car,Car,Car,Hydrant,Car,Car,Hydrant,Car
3,query11.png,Palm,Palm,Palm,Palm,Palm,Palm,Traffic Light,Traffic Light,Palm,Palm
4,query12.png,Traffic Light,Traffic Light,Palm,Traffic Light,Palm,Palm,Palm,Palm,Palm,Palm
...,...,...,...,...,...,...,...,...,...,...,...
95,query95.png,Palm,Car,Hydrant,Car,Palm,Hydrant,Car,Motorcycle,Car,Car
96,query96.png,Crosswalk,Crosswalk,Palm,Bridge,Crosswalk,Crosswalk,Bridge,Crosswalk,Crosswalk,Crosswalk
97,query97.png,Palm,Palm,Bus,Crosswalk,Palm,Palm,Palm,Palm,Traffic Light,Traffic Light
98,query98.png,Hydrant,Hydrant,Palm,Motorcycle,Car,Palm,Palm,Hydrant,Bicycle,Traffic Light


In [ ]:
#submit_classification.to_csv('/content/drive/Othercomputers/내 컴퓨터/Konkuk_ML/CV/model/SIFT_LBP_COS_300/c1_t1_a2.csv',index=False)

In [15]:
submit_ret.to_csv('/content/drive/Othercomputers/내 컴퓨터/Konkuk_ML/CV/model/SIFT_LBP_COS_300/c1_t2_a2.csv',index=False)